In [2]:
import pandas as pd
import numpy as np
import datetime
import sqlite3
from pandas.tseries.holiday import USFederalHolidayCalendar as uscal

import requests
import json
import glob
import gzip

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
%matplotlib inline

C:\Users\szhang\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
df = pd.read_csv('Mayfair Court Intervals.csv')

In [7]:
lat = 37.35396
lon = -121.85137
def data_sunrise(lat, lon, date):
    url = 'https://api.sunrise-sunset.org/json?lat={}&lng={}&date={}'.format(lat, lon, date)
    info = requests.get(url)
    new = json.loads(info.text)
    return new['results']['sunrise']

def data_sunset(lat, lon, date):
    url = 'https://api.sunrise-sunset.org/json?lat={}&lng={}&date={}'.format(lat, lon, date)
    info = requests.get(url)
    new = json.loads(info.text)
    return new['results']['sunset']

In [8]:
df['sunrise'] = 0
df['sunset'] = 0
df['start'] = pd.to_datetime(df['interval_start']).dt.date

In [9]:
for i in range(len(df)):

    
    df['sunrise'][i]= data_sunrise(lat, lon, df['start'][i])
    df['sunset'][i]= data_sunset(lat, lon, df['start'][i])

C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\szhang\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [10]:
df.head()

,meter_uid,utility,utility_service_id,utility_service_address,utility_meter_number,utility_tariff_name,interval_start,interval_end,interval_kWh,net_kWh,source,updated,interval_timezone,sunrise,sunset,start
0,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,B10S Bus Medium Use,05/22/2021 23:45,05/23/2021 00:00,4.48,4.48,NaN,NaN,US/Pacific,12:51:26 PM,3:16:48 AM,2021-05-22
1,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,B10S Bus Medium Use,05/22/2021 23:30,05/22/2021 23:45,4.32,4.32,NaN,NaN,US/Pacific,12:51:26 PM,3:16:48 AM,2021-05-22
2,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,B10S Bus Medium Use,05/22/2021 23:15,05/22/2021 23:30,4.64,4.64,NaN,NaN,US/Pacific,12:51:26 PM,3:16:48 AM,2021-05-22
3,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,B10S Bus Medium Use,05/22/2021 23:00,05/22/2021 23:15,4.64,4.64,NaN,NaN,US/Pacific,12:51:26 PM,3:16:48 AM,2021-05-22
4,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,B10S Bus Medium Use,05/22/2021 22:45,05/22/2021 23:00,5.12,5.12,NaN,NaN,US/Pacific,12:51:26 PM,3:16:48 AM,2021-05-22


In [11]:
df.to_csv('Mayfair.csv')

In [3]:
df = pd.read_csv('Mayfair.csv')

In [11]:
df.tail()

,Unnamed: 0,meter_uid,utility,utility_service_id,utility_service_address,utility_meter_number,utility_tariff_name,interval_start,interval_end,interval_kWh,...,sunset,start,sunrise_pst,sunset_pst,weekday,hour,date,Holiday,workday,season
36087,36087,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,A10SX Medium General Demand-Metered Service - TOU,05/11/2020 01:00,05/11/2020 01:15,5.76,...,2021-10-19 03:07:46,2020-05-11,05:59:45,20:07:46,0,1,2020-05-11,False,1,Spring
36088,36088,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,A10SX Medium General Demand-Metered Service - TOU,05/11/2020 00:45,05/11/2020 01:00,5.76,...,2021-10-19 03:07:46,2020-05-11,05:59:45,20:07:46,0,0,2020-05-11,False,1,Spring
36089,36089,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,A10SX Medium General Demand-Metered Service - TOU,05/11/2020 00:30,05/11/2020 00:45,6.08,...,2021-10-19 03:07:46,2020-05-11,05:59:45,20:07:46,0,0,2020-05-11,False,1,Spring
36090,36090,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,A10SX Medium General Demand-Metered Service - TOU,05/11/2020 00:15,05/11/2020 00:30,5.76,...,2021-10-19 03:07:46,2020-05-11,05:59:45,20:07:46,0,0,2020-05-11,False,1,Spring
36091,36091,764347,PG&E,5001388251,65 MCCREERY AVE,1009484062,A10SX Medium General Demand-Metered Service - TOU,05/11/2020 00:00,05/11/2020 00:15,6.08,...,2021-10-19 03:07:46,2020-05-11,05:59:45,20:07:46,0,0,2020-05-11,False,1,Spring


In [4]:
weather = pd.read_csv('weather_65.csv')

In [5]:
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])
df['sunrise_pst'] = pd.to_datetime(df['sunrise'] - pd.Timedelta(hours=7)).dt.time
df['sunset_pst'] = pd.to_datetime(df['sunset'] - pd.Timedelta(hours=7)).dt.time

In [6]:
df['weekday'] = pd.to_datetime(df['interval_start']).apply(lambda x: x.dayofweek)
df['hour'] = pd.to_datetime(df['interval_start']).dt.hour
df['date'] = pd.to_datetime(df['interval_start']).dt.date
cal = uscal()
holidays = cal.holidays(start=pd.to_datetime(df['date']).dt.date.min(), 
                        end=pd.to_datetime(df['date']).dt.date.max())
df['Holiday'] = df['date'].astype('datetime64').isin(holidays)
df['workday'] = np.where(df.weekday>4, 0, 1)
df['workday'] = np.where(df['Holiday']==1, 0, df['workday'])

In [7]:
df['season'] = ''
df.loc[pd.to_datetime(df['interval_start']).dt.month.isin([9,10,11]), 'season'] = 'Fall'
df.loc[pd.to_datetime(df['interval_start']).dt.month.isin([12,1,2]), 'season'] = 'Winter'
df.loc[pd.to_datetime(df['interval_start']).dt.month.isin([3,4,5]), 'season'] = 'Spring'
df.loc[pd.to_datetime(df['interval_start']).dt.month.isin([6,7,8]), 'season'] = 'Summer'

In [8]:
df_hour = pd.DataFrame(df.groupby(['date','hour', 
                'sunrise_pst', 'sunset_pst', 'weekday', 'Holiday', 'workday', 'season'])['interval_kWh'].sum()).reset_index()

In [9]:
df_hour['timestamp'] = pd.to_datetime(df_hour['date']) + pd.to_timedelta(df_hour['hour'], unit='h')

In [10]:
wed = weather[['dt_iso', 'temp', 'feels_like', 'pressure', 'humidity']]
wed['hour_before'] = wed['temp'].shift(periods = 1)
wed['timestamp'] = pd.to_datetime(wed.dt_iso.str.split('+').str[0])
wed['timestamp'] = pd.to_datetime(pd.to_datetime(wed.dt_iso.str.split('+').str[0]) - pd.Timedelta(hours=7))

C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\szhang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [12]:
wed_rang = wed[(wed['timestamp'] > '2020-05-10 00:00:00') & (wed['timestamp'] < '2021-05-23 01:00:00')].reset_index(drop=True)

In [13]:
wed_range = wed_rang.drop_duplicates(subset='timestamp', keep='first')
hours_wed = pd.merge(df_hour, wed_range, on='timestamp', how = 'left')

In [14]:
conn = sqlite3.connect(':memory:')
#write the tables
hours_wed.to_sql('data', conn, index=False)
qry = '''
select 
timestamp,interval_kWh, weekday, temp, hour, hour_before, feels_like, workday, season,
       pressure, humidity, sunrise_pst, sunset_pst, 
CASE
when time(timestamp) > time(sunrise_pst) and time(timestamp) < time(sunset_pst) THEN "Yes"
else "No"
END as Sunlight
from data

'''
hours_weather = pd.read_sql_query(qry, conn)

In [15]:
sunlight = pd.get_dummies(hours_weather.Sunlight, prefix = 'sunlight')
hours = pd.get_dummies(hours_weather.hour, prefix = 'hour')
workday = pd.get_dummies(hours_weather.workday, prefix = 'workday')
season = pd.get_dummies(hours_weather.season, prefix = 'season')
model_prep = hours_weather.join([sunlight, hours, workday, season])

In [16]:
model_prep.to_csv('Mayfair_all.csv')